In [1]:
import pandas as pd

# Load the data from the CSV file
file_path = 'gb.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to verify correct loading and to understand the structure of the data
data.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,London,51.5072,-0.1275,United Kingdom,GB,"London, City of",primary,11262000,8825001
1,Birmingham,52.4800,-1.9025,United Kingdom,GB,Birmingham,NaN,2919600,1137100
2,Portsmouth,50.8058,-1.0872,United Kingdom,GB,Portsmouth,NaN,855679,248440
3,Southampton,50.9025,-1.4042,United Kingdom,GB,Southampton,NaN,855569,271173
4,Nottingham,52.9561,-1.1512,United Kingdom,GB,Nottingham,NaN,729977,289301


In [8]:
# Select only the necessary columns and rename them for consistency
filtered_data = data[['city', 'lat', 'lng', 'population']].copy()
filtered_data.columns = ['City', 'Latitude', 'Longitude', 'Population']

# Sort the data by population in descending order and handle ties by keeping all tied entries
filtered_data_sorted = filtered_data.sort_values(by='Population', ascending=False)

# Get the top 50 cities, taking ties into account
# We need to include all cities that tie with the 50th city's population if there are any
population_cutoff = filtered_data_sorted.iloc[49]['Population']
top_cities = filtered_data_sorted[filtered_data_sorted['Population'] >= population_cutoff]

# Display the resulting top cities to confirm selection and count
top_cities.head(), top_cities.shape

(          City  Latitude  Longitude  Population
 0       London   51.5072    -0.1275    11262000
 1   Birmingham   52.4800    -1.9025     2919600
 2   Portsmouth   50.8058    -1.0872      855679
 3  Southampton   50.9025    -1.4042      855569
 4   Nottingham   52.9561    -1.1512      729977,
 (50, 4))

In [9]:
from scipy.spatial.distance import pdist, squareform
# Calculate the distance matrix for the cities
coords = top_cities[['Latitude', 'Longitude']]
distance_matrix = squareform(pdist(coords, metric='euclidean'))

In [26]:
class AntColonyOptimized:
    def __init__(self, distances, city_names, n_ants, n_best, n_iterations, decay, alpha=1, beta=2):
        """
        Initialize the Ant Colony Optimizer.
        """
        self.distances = distances
        self.city_names = city_names
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.construct_paths()
            self.spread_pheromone(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path
            self.pheromone *= self.decay  # correct pheromone decay
        return self.get_city_names(all_time_shortest_path[0]), all_time_shortest_path[1]

    def construct_paths(self):
        all_paths = []
        for _ in range(self.n_ants):
            path = self.construct_path(0)
            all_paths.append((path, self.path_cost(path)))
        return all_paths

    def construct_path(self, start):
        path = [start]
        visited = set(path)
        while len(path) < len(self.distances):
            move = self.probable_next_move(self.pheromone[path[-1]], self.distances[path[-1]], visited)
            path.append(move)
            visited.add(move)
        path.append(start)  # return to start
        return path

    def probable_next_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0
        # Ensure distances are not zero to prevent division by zero
        dist = np.where(dist == 0, 1e-10, dist)  # Replace 0 with a very small number
        row = pheromone ** self.alpha * ((1.0 / dist) ** self.beta)
        total = row.sum()
        if total == 0:
            norm_row = np.ones(len(row)) / len(row)  # Use uniform probabilities if sum is 0
        else:
            norm_row = row / total
        move = np.random.choice(self.all_inds, 1, p=norm_row)[0]
        return move

    def spread_pheromone(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in range(len(path) - 1):
                self.pheromone[path[move], path[move + 1]] += 1.0 / dist
                self.pheromone[path[move + 1], path[move]] += 1.0 / dist  # symmetric update

    def path_cost(self, path):
        return sum([self.distances[path[i], path[i + 1]] for i in range(len(path) - 1)])

    def get_city_names(self, path):
        return [self.city_names[i] for i in path]

# Instantiate and run the ant colony optimizer
ant_colony = AntColonyOptimized(distance_matrix, top_cities['City'].tolist(), n_ants=10, n_best=5, n_iterations=100, decay=0.95, alpha=1, beta=2)
shortest_path, shortest_distance = ant_colony.run()
shortest_path, shortest_distance


(['London',
  'Westminster',
  'Islington',
  'Tottenham',
  'Walthamstow',
  'Ilford',
  'Romford',
  'Croydon',
  'Harrow',
  'Slough',
  'High Wycombe',
  'Luton',
  'Worthing',
  'Portsmouth',
  'Southampton',
  'Basingstoke',
  'Oxford',
  'Northampton',
  'Leicester',
  'Nottingham',
  'Derby',
  'Rotherham',
  'York',
  'Bradford',
  'Rochdale',
  'Stockport',
  'Manchester',
  'Sale',
  'Warrington',
  'Liverpool',
  'Blackpool',
  'Blackburn',
  'Gateshead',
  'Aberdeen',
  'Belfast',
  'Plymouth',
  'Bournemouth',
  'Bristol',
  'Gloucester',
  'Wolverhampton',
  'West Bromwich',
  'Birmingham',
  'Solihull',
  'Coventry',
  'Cambridge',
  'Norwich',
  'Colchester',
  'Chelmsford',
  'Basildon',
  'Maidstone',
  'London'],
 34.58043477158881)

In [20]:
from math import radians, cos, sin, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the Haversine distance between two points on the earth (specified in decimal degrees)."""
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    r = 6371  # Radius of Earth in kilometers. Use 3956 for miles
    return c * r
    
def calculate_total_haversine_distance(route, coords):
    """ Calculate total Haversine distance of a route. """
    total_distance = 0
    for i in range(len(route)-1):
        lat1, lon1 = coords.iloc[route[i]][['Latitude', 'Longitude']]
        lat2, lon2 = coords.iloc[route[i+1]][['Latitude', 'Longitude']]
        total_distance += haversine_distance(lat1, lon1, lat2, lon2)
    # Add distance to return to the starting city
    lat1, lon1 = coords.iloc[route[-1]][['Latitude', 'Longitude']]
    lat2, lon2 = coords.iloc[route[0]][['Latitude', 'Longitude']]
    total_distance += haversine_distance(lat1, lon1, lat2, lon2)
    return total_distance
# Adjusting the route coordinates using the original 'top_cities' DataFrame that contains the coordinates of the cities
aco_route_coords = top_cities.iloc[aco_route_indices]

# Re-calculating the total Haversine distance using the correct coordinates of the cities in the route
total_haversine_distance_aco = calculate_total_haversine_distance(aco_route_indices, aco_route_coords)
total_haversine_distance_aco

11198.72623676193

In [1]:
x = 'teset'